# Basic Use Case: Question and Answering from a Document

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]


# Setup LLM to use, gemma using Groq

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7B-32768",
    temperature=0, # no hallucination 
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## Load The text article

In [4]:
from langchain_community.document_loaders import TextLoader

In [5]:
loader = TextLoader('data/be-good-and-how-not-to-die.txt')

In [6]:
document = loader.load()

## The document is loaded as a python list with metadada

In [7]:
print(type(document))
print(document[0].metadata)
print(f'You have {len(document)} document.')
print(f'Your document has {len(document[0].page_content)} characters.')

<class 'list'>
{'source': 'data/be-good-and-how-not-to-die.txt'}
You have 1 document.
Your document has 27423 characters.


#### Split the document in different chunks

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap=400)
document_chunks = text_splitter.split_documents(documents=document)

In [10]:
print(f'Now you have {len(document_chunks)} chunks.')

Now you have 12 chunks.


##  Convert text chunks in numeric vectors (Embeddings)

#### Using HuggingFace hosted all-MiniLM-L6-v2 Embeddings model 

In [11]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

c:\Users\luism\Documents\ai-school\ai-rag-document-qa\ai-rag-document-qa\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\luism\Documents\ai-school\ai-rag-document-qa\ai-rag-document-qa\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Create a Qdrant vector store collection

In [13]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore

qdrantClient = QdrantClient(url='localhost')
qdrantClient.create_collection(
    collection_name='rag-document',
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

### Ingest the chunked documents into the vector collection

In [14]:
stored_embeddings = QdrantVectorStore.from_documents(
    document_chunks,
    embeddings,
    url='localhost',
    collection_name="rag-document"
)

## Langchain Retrieval QA
We are using the prompts hub from lanchain you need to set the LANGCHAIN_API_KEY in the .env file

In [16]:
from langchain import hub

### Using the Vector Search Type similarity and return the 6 documents top relevant (k=6)

In [21]:
retriever = stored_embeddings.as_retriever(
        search_type="similarity",
        search_kwargs={'k': 6}
    )

prompt = hub.pull('rlm/rag-prompt')

### Join the retrieved documents from the vector database in a string

In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Build the rag chain using the new LCEL Runnable protocol
LangChain Expression Language (LCEL)
LangChain Expression Language, or LCEL, is a declarative way to chain LangChain components. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Invoke the chain, the argument is going to be the question as it is defined above RunnablePassthrough()

In [23]:
rag_chain.invoke("what are The Octoparts?")



'The Octoparts is a startup founded by two former physics PhD students from Berkeley. They created a search engine for electronic parts to solve a problem they encountered in their research. Despite the efforts of the largest distributor, Digi-Key, to force them to remove prices from the site, Octopart has continued to grow. This is because their goal is to help users by providing a valuable service, which acts as a compass for their decision-making. They are committed to succeeding and have transformed from lighthearted to grimly determined due to the challenges they face from electronic parts distributors.'